### For Gene_Data
[This](https://dataguide.nlm.nih.gov/eutilities/utilities.html) has been extremely useful, especially the part about efetch and the tables it links (which are very hard to find).

We will need two separate Entrez query methods:
    1. One for normally annotated species
    2. Another for metagenomes of newly added species
Both should return an xml which will be parsed differently according the !DOCTYPE tag (**Entrezgene-Set** or **GBSet**)

The default setting for *retmax* is 20 which was limiting our geneID results. The maximum value that can be used is 100000. For higher numbers we need to use different approach, found [here](https://www.ncbi.nlm.nih.gov/books/NBK25499/). But I don't think it will be necessary since none of our species will have that many genes.

Tags leading to the Gene ID:

    <Entrezgene-Set>
    <Entrezgene>
      <Entrezgene_track-info>
        <Gene-track>
          <Gene-track_geneid>38094981</Gene-track_geneid>

Tags leading to the gene name:

    <Entrezgene_gene>
        <Gene-ref>
            <Gene-ref_locus>ytiA</Gene-ref_locus>

Gene description and protein accession:

    <Gene-commentary_products>
        <Gene-commentary>
          <Gene-commentary_type value="peptide">8</Gene-commentary_type>
          <Gene-commentary_label>uncharacterized protein YtiA</Gene-commentary_label>
          <Gene-commentary_accession>YP_009518833</Gene-commentary_accession>
          <Gene-commentary_version>1</Gene-commentary_version>

There are several other tags that would be of interest, like chromosome, sequence interval from-to, strand direction.

<span style="color:red">There are entries from multiple databases within &lt;Entrezgene&gt;, defining the same gene.</span> Ideally we are to use the reference ones &lt;Gene-ref&gt;, &lt;Prot-ref&gt;, &lt;Entrezgene_locus&gt;.

In [ ]:
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"  # Always tell NCBI who you are
handle = Entrez.esearch(db="gene", term="txid511145[Organism]",retmax=1)
record = Entrez.read(handle)
handle.close()
print(record)
handle = Entrez.efetch(db="gene", id=record['IdList'],rettype="xml")
genes=handle.read()

print(genes)

### For new species/metagenomes
Proteins for species, determined from environmental samples are not present in the *gene* NCBI database. However, their protein sequences have been determined and exist in the *protein* database. We can use that to extract gi numbers, symbols, and descriptions.

    <GBSeq_definition>Chain A, Uncharacterized protein</GBSeq_definition>
    <GBSeq_primary-accession>5YEE_A</GBSeq_primary-accession>
    <GBSeq_accession-version>5YEE_A</GBSeq_accession-version>
    <GBSeq_other-seqids>
      <GBSeqid>pdb|5YEE|A</GBSeqid>
      <GBSeqid>gi|1482416786</GBSeqid>
    </GBSeq_other-seqids>           <----- gi is more important than the tag in htis case


In [1]:
handle = Entrez.esearch(db="protein", term="txid1538547[Organism]")   #Lokiarchaeum
record = Entrez.read(handle)
handle.close()
print(record)
handle = Entrez.efetch(db="protein", id=record['IdList'],rettype="gp", retmode="xml")
genes=handle.read()

print(genes)

NameError: name 'Entrez' is not defined

# Gene_Data table 
This table contains information regarding the GeneID, StrainID, the nomgd_ID(which is the nomenclature ID generated in the nomenclature table), gene symbol and gene description. It gets the strain ID from the species table and the gene information such as the gene ID, gene symbol and gene description from NCBI database.

We need to build functions for cases where there are no hits for the query i.e., if we don't find any gene information for a particular strain ID.

There are three strainIDs that do not have any entries in the gene database for bacteria. So, these three strain IDs showed up in the output as no records exist :

Record doesn't exist 165597 - Crocosphaera watsonii WH 8501;
Record doesn't exist 272627 - Magnetospirillum magnetotacticum MS-1;
Record doesn't exist 575564 - Acinetobacter sp. RUH2624.

In [4]:
#! /usr/bin/env python3

from Bio import Entrez
from xml.dom.minidom import parse, parseString
import xml.dom.minidom as minidom
import subprocess
import re
import mysql.connector
import getpass

uname = input("User name: ")
pw = getpass.getpass("Password: ")
cnx = mysql.connector.connect(user=uname, password=pw, host='130.207.36.75', database='DESIREE')
cursor = cnx.cursor()

def get_strainid():
	'''
	function to get a list of strain IDs from the TaxGroup table
	'''
	strainID_list = []
	cursor.execute("SELECT DESIRE.Species.strain_id FROM DESIRE.Species_TaxGroup\
				INNER JOIN DESIRE.TaxGroups ON DESIRE.Species_TaxGroup.taxgroup_id=DESIRE.TaxGroups.taxgroup_id\
				INNER JOIN DESIRE.Species ON DESIRE.Species_TaxGroup.strain_id=DESIRE.Species.strain_id\
				WHERE DESIRE.TaxGroups.groupName = 'Bacteria'")
	results = cursor.fetchall()
	for i in results:
		strainID_list.append(int(i[0]))
	#print(len(strainID_list))
	return strainID_list

def superkingdom_info(ID):
	'''
	function to get the superkingdoms for the strain IDs
	'''
	#print(ID)
	cursor.execute("SELECT DESIRE.TaxGroups.groupName FROM DESIRE.Species_TaxGroup\
		INNER JOIN DESIRE.TaxGroups ON DESIRE.Species_TaxGroup.taxgroup_id=DESIRE.TaxGroups.taxgroup_id\
		INNER JOIN DESIRE.Species ON DESIRE.Species_TaxGroup.strain_id=DESIRE.Species.strain_id\
		WHERE DESIRE.TaxGroups.groupLevel = 'superkingdom' AND DESIRE.Species.strain_id = '"+ID+"'")
	results = cursor.fetchall()
	superkingdom=(results[0][0])
	#print(superkingdom)
	return superkingdom

def nomid_oldname(domain_letter):
	'''
	function to get the nomo_id and old_name for the superkingdom obtained from superkingdom_info function
	'''
	cursor.execute("SELECT DESIRE.Old_name.nomo_id, DESIRE.Old_name.old_name FROM DESIRE.Old_name\
		INNER JOIN DESIRE.Nomenclature ON DESIRE.Old_name.nomo_id=DESIRE.Nomenclature.nom_id\
		WHERE DESIRE.Nomenclature.occurrence = '"+domain_letter+"' AND DESIRE.Old_name.N_B_Y_H_A = '"+domain_letter+"'")
	oldname = cursor.fetchall()
	# print(oldname)
	return oldname

def gene_info(taxid):
	'''
	function to get gene_id, gene_symbol, and gene_desc for the Gene_Data table using the strain ID
	'''
	#print(taxid)
	geneinfo_output = subprocess.check_output("esearch -db gene -query 'txid"+taxid+"[Organism] ribosomal' | efetch -format tabular", shell=True)
	#print(geneinfo_output)
	gene_out = geneinfo_output.decode('utf-8')
	# print(gene_out)
	if re.match(".*https\:\/\/eutils\.st-va\.ncbi\.nlm\.nih\.gov\.*", gene_out):
		print("There is an error")
		pass
	elif re.match(".*https\:\/\/eutils\.be-md\.ncbi\.nlm\.nih\.gov\.*", gene_out):
		print("There is an error")
		pass
	else:
		# print(gene_out)
		gene_out = gene_out.split('\n')
		regex = re.compile('^tax_id\\tOrg_name\\tGeneID\\t.*')
		gene_out = [i for i in gene_out if not regex.match(i)] 
		regex = re.compile('.*NEWENTRY.*')
		gene_out = [i for i in gene_out if not regex.match(i)]
		# print(gene_out)
		genesym_desc=[]
		for item in gene_out[:-2]:
			gene_list=item.split('\t')
			if gene_list[7] == 'pseudo' or gene_list[7] == 'hypothetical protein':
				pass
			else:
				geneinfo = (gene_list[2], gene_list[5], gene_list[7]) # gene_id, gene_symbol, gene_desc
				# print(geneinfo)
				genesym_desc.append(geneinfo)
		#print(genesym_desc)
		return genesym_desc

def upload_data(name, desc, strainID):
	for i in name:
		old_name = i[1]
		nom_id=i[0]
		for j in desc:
			word = j[2]
			regexp = re.compile(r''+old_name+'$')
			regexp_RNA = re.compile(r'ribosomal RNA$')
			if regexp.search(word): 
				query = ("INSERT INTO `DESIREE`.`Gene_Data`(`GI`,`strain_ID`,`nomgd_id`, `GeneSymbol`, `GeneDescription`) VALUES('"+str(j[0])+"','"+str(strainID)+"','"+str(i[0])+"','"+str(j[1])+"','"+str(j[2])+"')")
				cursor.execute(query)
				#print(query)
			if regexp_RNA.search(word):
				query = ("INSERT INTO `DESIREE`.`Gene_Data`(`GI`,`strain_ID`, `GeneSymbol`, `GeneDescription`) VALUES('"+str(j[0])+"','"+str(strainID)+"','"+str(j[1])+"','"+str(j[2])+"')")
				cursor.execute(query)
				#print(query)


strainID_list=get_strainid()
	
for strainID in strainID_list:
	genesym_desc=gene_info(str(strainID))
	
	if len(genesym_desc) == 0:
		pass #change this - use another function to access protein db rather than gene db
		print("Record doesn't exist", strainID)

	superkingdom=superkingdom_info(str(strainID))
	nom_oldList = nomid_oldname(str(superkingdom[0]))
	upload_data(nom_oldList, genesym_desc, strainID)
cnx.commit()
cursor.close()
cnx.close()

User name: vchivukula
Password: ········
Record doesn't exist 165597
Record doesn't exist 272627
Record doesn't exist 575564
